# Convex optimization in Python

This is the first of two exercise notebooks on conic programming for machine
learning.

All exercises have a theoretical and a practical part, the latter using Python
and the PICOS library to model and solve optimization problems. Very basic
knowledge of Python is assumed, experience with PICOS or a similar library
(notably CVXPY or Pyomo under Python and Convex.jl under Julia) is not.

These notebooks are meant to be **solved in a small group** and with pen and
paper next to the keyboard. Have fun!

## Preamble

The code below installs the *Python Interface to Conic Optimization Solvers*,
[PICOS](https://picos-api.gitlab.io/picos/), and its minimum dependencies (NumPy
and CVXOPT), alongside other packages that we use in the notebooks.

Installation via pip is recommended and should work on GNU/Linux and Mac OS as
long as Python `3.4` or later is installed and set as the kernel for this
notebook. If you use Python through conda (which is more common on Windows),
then you need to run this notebook in a conda environment where you can install
packages using the alternative commands below (but possibly also via pip).

In [ ]:
# Install using pip (recommended).
# NOTE: On certain platforms you may need to add --break-system-packages to
#       allow installation to ~/.local/lib/ when not running in a venv. This is
#       indicated by an "error: externally-managed-environment" message.
%pip install --user -r requirements.txt

# Install using conda (uncomment to enable; requires active conda environment).
# %conda install -c conda-forge ipywidgets
# %conda install -c picos picos
# %conda install -c plotly plotly
# %conda install -c anaconda scikit-image scipy

# Configure plotly to work inside a notebook.
# NOTE: If this fails after installing plotly, restart the notebook kernel.
from plotly.offline import init_notebook_mode
init_notebook_mode()

### Installing additional solvers (optional)

PICOS is not itself a numeric solver. Instead, it provides a modeling framework
and a unified interface to many free and commercial solvers. You already have
[CVXOPT](https://cvxopt.org/) installed as a baseline solver, which handles all
continuous problem types that can be defined within PICOS but not integer
problems.

To support also mixed integer programming or to get improved performance for
large problems, additional solvers can be installed and will be used by PICOS as
appropriate. All of them can be installed via pip, though some require a license
to run, or are limited trial versions:

In [ ]:
# Free solvers:
# %pip install ecos
# %pip install osqp
# %pip install swiglpk

# Interface only:
# %pip install pyscipopt  # requires installation of SCIP

# Commercial solvers:
# %pip install cplex      # comes with a limited license
# %pip install gurobipy   # comes with a limited license
# %pip install mosek      # requires a license

<details>
<summary style='display: list-item'>Solver capabilities</summary>

- [CPLEX](https://www.ibm.com/de-de/analytics/cplex-optimizer) is a classic
  commercial solver that solves (integer) linear programs ((I)LPs) and
  (quadratically constrained) quadratic programs ((QC)QPs).
- [ECOS](https://github.com/embotech/ecos) is a free solver designed for
  embedded devices that offers support for mixed integer LPs and second-order
  conic programs (SOCPs).
- [GLPK](https://www.gnu.org/software/glpk/) is a free but aged LP and ILP
  solver from the GNU project. It has an actively maintained third-party Python
  interface in [swiglpk](https://github.com/biosustain/swiglpk).
- [Gurobi](https://www.gurobi.com/) is a spiritual successor to CPLEX with a
  similar scope.
- [MOSEK](https://www.mosek.com/), like CVXOPT, supports all cones used by PICOS
  and is most popular for solving SDPs.
- [OSQP](https://osqp.org/) is free and solves LPs and QPs (but not QCQPs).
- [SCIP](https://scipopt.org/) is a free (since version `8.0.3`) solver with a
  focus on integer LPs and (QC)QPs. It has a Python interface in
  [PySCIPOpt](https://github.com/scipopt/PySCIPOpt) that needs to be installed
  separately.

More details on solver capabilities can be found
[here](https://picos-api.gitlab.io/picos/introduction.html#features).
  
</details>

As a member of an academic institution, you can get full-featured licenses for
the commercial solvers free of charge.

### Confirming the installation

Let us now see whether we can import PICOS and what solvers are detected by it:

In [ ]:
import picos as pc

# List all solvers that are detected.
pc.available_solvers()

The code above should return a list of solver names containing at least
`'cvxopt'`.

## Problem 1: Constrained linear regression

### Setting

To get started we solve a simple geometric problem: we project an
$m$-dimensional point $b$ onto the [convex
hull](https://en.wikipedia.org/wiki/Convex_hull) of a set of points $\{a_1,
\ldots, a_n\}$, which are given as the columns of a matrix $A = \begin{bmatrix}
a_1 & \cdots & a_n \end{bmatrix}$. We will store this data in NumPy arrays,
which are the most widely used format for numerical computation with Python:

In [ ]:
import numpy as np

# Uncomment to make the data reproducible.
# np.random.seed(1)

# Set m and n. Note that only m = 2 and m = 3 can be plotted.
m = 3
n = 40

# Define a random m×n matrix A and an m-vector b.
A = np.random.rand(m, n)
b = np.array([1]*m)

Using SciPy and Plotly, we can compute the convex hull of the $a_i$ and draw it:

In [ ]:
from scipy import spatial
from plotly import graph_objects as go

# Style the figure.
LAYOUT = dict(
    scene=dict(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False),
    yaxis=dict(scaleanchor="x1"), margin=dict(l=10, r=10, t=10, b=10),
    paper_bgcolor="white", plot_bgcolor="rgba(0,0,0,0)",
    height=600,
)

# Compute the vertices on the convex hull.
H = spatial.ConvexHull(A.T)
V = H.vertices
F = H.simplices

print(f"{len(V)}/{n} vertices are in convex position.")

# Plot the points and the convex hull.
if m in (2, 3):
    style_1 = dict(mode="markers", name="A", marker_color="black")
    style_2 = dict(mode="markers", name="b", marker_color="red")

    if m == 2:  # 2D plot
        C = np.concatenate([V, [V[0]]])
        traces = [
            go.Scatter(
                x=A[0,C], y=A[1,C], mode="none", fill="toself",
                name="conv(A)", fillcolor="lightgray"
            ),
            go.Scatter(x=A[0,:], y=A[1,:], **style_1),
            go.Scatter(x=[b[0]], y=[b[1]], **style_2)
        ]
    else:  # 3D plot
        style_1["marker_size"] = style_2["marker_size"] = 5
        edges = dict(zip("xyz", [
            [
                A[i, F[j][k % len(F[j])]] if k <= len(F[j]) else None
                for j in range(len(F)) for k in range(len(F[j]) + 2)
            ]
            for i in range(3)
        ]))
        traces = [
            go.Mesh3d(
                x=A[0,V], y=A[1,V], z=A[2,V], color="gray", opacity=0.5,
                alphahull=0, flatshading=True, name="conv(A)",
            ),
            go.Scatter3d(
                **edges, mode="lines", line_color="black", opacity=0.1,
                showlegend=False, hoverinfo="skip"
            ),
            go.Scatter3d(x=A[0,:], y=A[1,:], z=A[2,:], **style_1),
            go.Scatter3d(x=[b[0]], y=[b[1]], z=[b[2]], **style_2)
        ]
    
    go.Figure(traces, LAYOUT).show()
else:
    print("Can only plot 2D and 3D scenarios.")

You should now see an interactive sketch of the geometric setting above this
line. (You can drag & drop to change the camera perspective.)

### The problem

The projection of $b$ onto $\operatorname{conv}(A) =
\operatorname{conv}\left(\{a_1, \ldots, a_n\}\right)$ is the unique point in the
convex hull of the $a_i$ that is closest to $b$ under the Euclidean norm. Thus,
we are looking to solve a minimization problem:

$$
\begin{align*}
    \text{minimize} ~&~ \lVert p - b \rVert \tag{1} \\
    \text{subject to} ~&~ p \in \operatorname{conv}(A).
\end{align*}
$$

This is a convex optimization problem as we are minimizing a norm over a (by
definition) convex domain. That's good news, as this typically means that the
problem is tractable! If we want to use existing software to solve it, though,
we can expect that there will be no direct representation for
$\operatorname{conv}(A)$ available with the software's interface. Then, we have
to reformulate the problem.

---

<div class="alert alert-info">

   **Task 1.1:** Rewrite optimization problem (1) to use only the Euclidean
   norm, [affine functions](https://mathworld.wolfram.com/AffineFunction.html),
   and element-wise (in)equalities between vectors. (Edit the cell below.)

   <details>
   <summary style='display: list-item'>Hint 1</summary>

   - Every point in $\operatorname{conv}(A)$ can be written as a [convex
     combination](https://en.wikipedia.org/wiki/Convex_combination) of the
     columns of $A$.

   </details>

   <details>
   <summary style='display: list-item'>Hint 2</summary>

   - You will need to introduce a new variable, say $x$. It can either replace
     the variable $p$, in which case $p$ may be recovered from a solution for
     $x$, or it can occur alongside $p$ in the problem.

   </details>
</div>

<div hidden>

$$
\providecommand{\TODO}{}\renewcommand{\TODO}{{\color{red}{[\ldots]}}}
$$

</div>

$$
\begin{align*}
    \text{minimize} ~&~ \TODO \tag{2} \\
    \text{subject to}
    ~&~ \TODO, \\
    ~&~ \TODO. \\
\end{align*}
$$


---

<div class="alert alert-info">

   **Task 1.2:** Complete the code below to obtain a numeric solution for $p$.

</div>

Let's implement problem (2) in PICOS. The first step is optional: we load the
NumPy data using `Constant` to give it symbolic names.

In [ ]:
# Load the matrix A as a PICOS constant (to assign a name to it).
A = pc.Constant("A", A)

# TODO: Load also the vector b.


We can inspect the constants using `print`.

In [ ]:
print(repr(A))  # representation for debugging
print(A)        # string for regular output

print(repr(b))
print(b)

Note that the NumPy $m$-vector was loaded as a $m \times 1$ column vector. While
NumPy supports tensors of any order, in PICOS every constant is a matrix. The
transpose of a matrix is denoted by `.T`:

In [ ]:
print(repr(A.T))
print(repr(b.T))  # b as a row vector.

Next, we define the variable(s) that we want to optimize over. For real-valued
variables we may use the `RealVariable` class. You can look up its documentation
[online](https://picos-api.gitlab.io/picos/api/picos.expressions.variables.html#picos.expressions.variables.RealVariable)
or use Python's built-in `help` function:

In [ ]:
# Display information on how to construct a RealVariable.
# NOTE: C.__init__ is called when you write C(...) for a Python class C.
help(pc.RealVariable.__init__)

In [ ]:
# TODO: Define a variable or variables with suited name and shape.


# TODO: Inspect the created variable(s).


Constants and variables are the building blocks of more complex expressions that
occur in an optimization problem. For matrices $A$ and $B$ and a column vector
$c$ of suited shapes, you can write, for instance:

| on paper | in picos |
| --- | --- |
| $A + B$ | `A + B` |
| $A B$ | `A*B` |
| $c^T A$ | `c.T*A` |
| $\lVert c \rVert_2$ | `abs(c)` |
| $\lVert A \rVert_F$ | `abs(A)` |
| $\sum_{ij} A_{ij}$ | `A.sum` |
| $A = B$ | `A == B` |
| $A \leq B$ | `A <= B` |
| $A \geq B$ | `A >= B` |

The (in-)equalities are meant entry-wise. The norms are $\lVert c
\rVert_2=\sqrt{\sum_{i} c_i^2}$ and $\lVert A \rVert_F=\sqrt{\sum_{ij}
A_{ij}^2}$. Armed with these operations we can now define problem (2):

In [ ]:
# Create an optimization problem.
P = pc.Problem("Projection onto convex hull")

# TODO: Assign the objective function to P.minimize.
# P.minimize = 

# TODO: Add constraints using the in-place addition operator.
#       Syntax example: P += A*x == b
# P += 
# P += 

# Print the problem!
print(P)

You can now see why it is worthwhile to assign symbolic names to data via
`Constant`: it makes debugging your program a lot easier.

If the problem looks good, let's obtain a solution using `Problem.solve`. The
`verbosity=1` argument shows the reformulations done by PICOS and the solver's
progress.

In [ ]:
# Solve the problem.
# NOTE: You can add, e.g., solver="cvxopt" to select a solver manually.
solution = P.solve(verbosity=1)

If all went well, the `solution` variable now contains a numeric solution and
metadata provided by the solver. We often don't need it, as by default, the
solution found is applied to the problem variables.

In [ ]:
# Show the solution and the problem's variables.
print(f"solution:            {solution}")
print(f"objective is valued: {P.valued}")
print(f"objective value:     {P.value:.4f}\n")

for name, var in P.variables.items():
    print(f"{name} is valued:  {var.valued}")
    print(f"value of {name}.T: {var.T.value}", end="") # value as CVXOPT matrix

We can now obtain the projection point $p$ in NumPy format using the `.np`
property on either the PICOS variable representing $p$ or on another expression,
depending on how you defined the problem.

In [ ]:
# TODO: Obtain the projection point p as a NumPy array.
# p_np = 

# We expect an m-dimensional array:
print(repr(p_np))

Finally, let's plot the result!

In [ ]:
if m in (2, 3):
    style = dict(
        mode="markers+lines+text", name="projection", text=["b", "p"],
        marker_color="red", line_color="black", textposition="top center"
    )

    if m == 2:
        data = dict(x=[b[0], p_np[0]], y=[b[1], p_np[1]])
        trace = go.Scatter(**data, **style)
    else:
        style["marker_size"] = 5
        data = dict(x=[b[0], p_np[0]], y=[b[1], p_np[1]], z=[b[2], p_np[2]])
        trace = go.Scatter3d(**data, **style)

    if trace:
        go.Figure(traces[:-1] + [trace], LAYOUT).show()
else:
    print("Can only plot 2D and 3D scenarios.")

You can run this notebook again to get a new problem instance, or change the
dimensionality and number of points to check that your implementation
generalizes.

### Bonus: Low level solution

Sometimes, using PICOS or a similar modeling language is not possible, for
instance when the application is written in a low-level language like C or when
a large number of small problems must be solved, where the overhead becomes
noticeable. In these instances, it can be useful to know how to use a solver's
interface directly. It is not uncommon that this requires merging the
constraints into just one or two constraint matrices. For example, the direct
interface to CVXOPT accepts conic problems of the form

$$
\begin{align*}
    \text{minimize} ~&~ c^T x \tag{3} \\
    \text{subject to}
        ~&~ Gx \leq_K h, \\
        ~&~ Px = q,
\end{align*}
$$

where $K = K_1 \times \cdots \times K_\ell$ is a direct product of cones (recall
that $Gx \leq_K h \Longleftrightarrow h - Gx \in K$). The problem is thus input
as a collection of matrices $G$ and $P$, vectors $c$, $h$, and $q$, and a
definition of the product cone $K$.

---

<div class="alert alert-info">

**Bonus task 1.3:** Rewrite problem (1) into form (3), where $K =
\mathbb{R}_{\geq 0}^n \times \mathcal{Q}^{m+1}$ with $\mathcal{Q}^{m+1} = \{(t,
y) \in \mathbb{R} \times \mathbb{R}^{m} \mid t \geq \lVert y \rVert\}$.

<details>
<summary style='display: list-item'>Hint 1</summary>

We start with the following solution to Task 1.1:

$$
    \begin{align*}
        \text{minimize} ~&~ \lVert Ax - b \rVert \\
        \text{subject to}
        ~&~ \sum_{i=1}^n x_i = 1, \\
        ~&~ x \geq 0. \\
    \end{align*}
$$

Note further:

- The point $p = Ax$ can be obtained after solving for $x$, so we do not need to
  have a variable $p$ in the problem.
- $\sum_{i=1}^n x_i$ can also be written as the inner product $\mathbf{1}^T x$,
  where $\mathbf{1}$ denotes a vector of ones with same shape as $x$.

</details>

<details>
<summary style='display: list-item'>Hint 2</summary>

Use an *epigraph reformulation*: the problem
$$
    \text{minimize}~f(x)
$$
is equivalent to
$$
    \begin{align*}
        \text{minimize} ~&~ t \\
        \text{subject to} ~&~ f(x) \leq t,
    \end{align*}
$$
where $t \in \mathbb{R}$ is a new scalar variable.

</details>

<details>
<summary style='display: list-item'>Hint 3</summary>

To make use of the second-order cone $\mathcal{Q}^{m+1}$, replace $Ax - b$ under
the norm with another new variable $y \in \mathbb{R}^m$ and add the constraint
$y = Ax - b$.

</details>
</div>

<div hidden>

$$
\providecommand{\TODO}{}\renewcommand{\TODO}{{\color{red}{\mathbf{?}}}}
$$

</div>

$$
\begin{align*}
    \text{minimize} ~&~
    \begin{bmatrix}
        0, 1, 0
    \end{bmatrix}
    \begin{bmatrix}
        x \\
        t \\
        y
    \end{bmatrix} \tag{4} \\
    \text{subject to}
    ~&~
    \begin{bmatrix}
        \TODO & \TODO & \TODO \\
        0 & -1 & 0 \\
        0 & 0 & -I \\
    \end{bmatrix}
    \begin{bmatrix}
        x \\
        t \\
        y
    \end{bmatrix}
    \leq_K
    \begin{bmatrix}
    \TODO \\
    \TODO \\
    \TODO
    \end{bmatrix}, \\
    ~&~
    \begin{bmatrix}
        \mathbf{1}^T & 0 & 0 \\
        \TODO & 0 & \TODO \\
    \end{bmatrix}
    \begin{bmatrix}
        x \\
        t \\
        y
    \end{bmatrix}
    =
    \begin{bmatrix}
    1 \\
    \TODO
    \end{bmatrix}, \\
    \text{where}
    ~&~ x \in \mathbb{R}^n, \quad t \in \mathbb{R}, \quad y \in \mathbb{R}^m.\\
\end{align*}
$$

Note: We write $I$ for an identity matrix of adequate size, $\mathbf{1}$ for an
all-ones column vector and $\mathbf{1}^T$ for its transpose, an all-ones row
vector.


---

<div class="alert alert-info">

**Bonus task 1.4:** Implement problem (4) using CVXOPT's
[conelp](https://cvxopt.org/userguide/coneprog.html#cvxopt.solvers.conelp)
function.

<details>
<summary style='display: list-item'>Notes on notation differences</summary>

- The names $A$ and $b$ that occur in the definition of `conelp` correspond to
  $P$ and $q$ in problem (3). They should not be confused with the $A$ and $b$
  of problem (1). In fact, the $A$ and $b$ of problem (1) might occur in the
  definitions of $P$ and $q$, which we will then pass to the `A` and `b`
  arguments of `conelp`.
- The constraint $Gx \leq_K h$ of problem (3) is written as the two constraints
  $Gx + s = h$ and $s \succeq 0$ in the documentation of `conelp`. These are
  equivalent: $s$ plays the role of a [slack
  variable](https://en.wikipedia.org/wiki/Slack_variable) and $s \succeq 0$ is
  just a different notation for $s \geq_K 0$ or, equivalently, for $s \in K$.

</details>

<details>
<summary style='display: list-item'>Documentation for NumPy functions</summary>

Use

- [hstack](https://numpy.org/doc/stable/reference/generated/numpy.hstack.html),
  [vstack](https://numpy.org/doc/stable/reference/generated/numpy.vstack.html),
  and
  [block](https://numpy.org/doc/stable/reference/generated/numpy.block.html)
  to assemble vectors and block matrices,
- [eye](https://numpy.org/doc/stable/reference/generated/numpy.eye.html) for
  the identity matrix,
- [zeros](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html)
  and [ones](https://numpy.org/doc/stable/reference/generated/numpy.ones.html)
  to create all-one and all-zero arrays,
- `@` to denote matrix multiplication (`*` is used for element-wise
  multiplication!), and
- `a[i:j]` on a vector `a` to get the subvector from the $i$-th element
  (inclusive) to the $j$-th element (exclusive).

The following could also be useful:

- Write `print(A.shape)` to inspect the number of dimensions and shape of a
  NumPy array `A`, and
- `b[:, np.newaxis]` to convert a 1D array `b` into a 2D column vector.
  (Otherwise, NumPy treats 1D arrays like row vectors.)

</details>
</div>

In [ ]:
import cvxopt
from cvxopt.solvers import conelp

# When this cell first runs, A and b are still PICOS constants, so convert them
# back to NumPy arrays.
try:
    A, b = A.np, b.np
except AttributeError: # Already converted.
    assert isinstance(A, np.ndarray) and isinstance(b, np.ndarray)

# TODO: Define the data of problem (3) according to problem (4).
#       To this end use A, b, and np.block/vstack/hstack/eye/zeros/ones.
# c = 
# G = 
# h = 
# P = 
# q = 

# TODO: Define the cone K via a dictionary of dimensions.
#       (See the documentation for conelp linked above!)
dims = dict(
    # l=,
    # q=,
    # s=,
)

# Convert the NumPy arrays to CVXOPT matrices and call conelp.
data = dict(c=c, G=G, h=h, A=P, b=q)
data = {arg: cvxopt.matrix(val) for arg, val in data.items()}
solution = cvxopt.solvers.conelp(**data, dims=dims)

# TODO: Recover x and p from the solution.
z = np.ravel(np.array(solution["x"]))  # convert back from CVXOPT to NumPy
# x_cvx = 
# p_cvx = 

# If all went well, p_np from the solution via PICOS and p_cvx should be close.
close = np.allclose(p_np, p_cvx)
print(f"\np from PICOS:  {p_np.T}\np from CVXOPT: {p_cvx.T}")
print(f"almost equal:  {close}{r' - Congratulations!' if close else ''}")

The last two tasks show why modeling interfaces like PICOS are popular: the
performance gain of interfacing a solver directly typically comes at a larger
development cost and the resulting code is harder to understand and modify.